In [ ]:
### Start position 0
def data_generator(ix_train = 1080, ix_val = 256, ix_test = 128, data_size=1464):
        filelist = glob.glob("/scratch/maghoi/pMHC_data/features3/*.csv"); len(filelist)
        
        filelist_loaded = []
        ix_test = data_size - ix_test
        ix_val = ix_test - ix_val
        ix_train = ix_val - ix_train
        
        #Load data into dfs
        for i in range(0, len(filelist)):
            df = pd.read_csv(filelist[i])
            df = df.drop(["Unnamed: 0", "Num", "pACC", "CNb"], axis = 1)
            
            #Convert categorical values (res + solvent acc cateogry) to string
            #df.iloc[:, 0:2] = df.iloc[:, 0:2].astype(int).astype(str)
            filelist_loaded.append(df)

        #Initialize empty df ordered by complexes and aminos
        dim1 = range(0, data_size)
        dim2 = filelist_loaded[0].index.values
        dim3 = filelist_loaded[0].columns
        x = np.zeros(shape = (len(dim1), len(dim2), len(dim3)))

        for i in range(0, data_size):
            x[i] = filelist_loaded[i]

        #Encode y fresh
        y = np.zeros(shape = (data_size), dtype="int64")

        counter_x = range(0, data_size)
        counter_y = range(len(y))

        for c_x, c_y in zip(counter_x, counter_y):
            r = re.compile(r'.*P1.*')
            if bool(r.match(filelist[c_x])):
                y[c_y] = 1
                
        X_train, y_train = x[0 : ix_val], y[0 : ix_val]
        X_val, y_val = x[ix_val : ix_test], y[ix_val : ix_test]
        X_test, y_test = x[ix_test : ], y[ix_test : ]
        return X_train, y_train, X_val, y_val, X_test, y_test


In [ ]:
#input 405x5

# conv
# maxpool
# FF

In [ ]:
tensor([[-7.8191, -2.3250, -5.0659, -1.3401, -2.2402]], grad_fn=<ViewBackward>)
(Pdb) next
> <ipython-input-68-f9315860c2c2>(33)<module>()
-> loss.backward()
(Pdb) loss
tensor(7.0699, grad_fn=<NllLossBackward>)
(Pdb) y_.shape
torch.Size([1, 5])



In [ ]:
RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

> /home/maghoi/.conda/envs/fastai_py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py(3287)run_code()
-> return outflag

In [ ]:
batch_size = 1
my_data_bunch = DataBunch.create(train_ds,valid_ds,test_ds,bs=batch_size)
criterion = nn.CrossEntropyLoss() 

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv1d(in_channels = 405, out_channels = 1, kernel_size = 1)
        #self.conv1 = nn.Conv1d(in_channels = 405, out_channels = 1, kernel_size = 1)
        #self.maxpool = nn.MaxPool2d(kernel_size = 1, dilation = 1)
        #self.softmax = nn.Softmax(dim = 1)
        #self.conv1 = nn.Linear(1*9, out_features=1)
        #self.conv2 = nn.Linear(1*1, out_features=1)
        #self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        input_size = x.shape[0]
        print("Input", x.shape)
        
        #Re-shape 405x5 to 5x405 to train on amino acids
        #x.reshape(input_size, 5, 405)
        x = self.conv1(x)
        print("After conv:", x.shape)
        x = x.reshape(input_size, 5)
        #x = self.softmax(x)
        #x = F.relu(self.conv1(x))
        #x = F.relu(self.conv2(x))
        print("\nEnd:", x.shape)
        print("End:", x)
        return x

#print(net)  
#print( 'trainable params: {}'.format(count_model_params(net)) )
net = Model().cuda()

my_learner = Learner(my_data_bunch,
                     net,
                     opt_func=torch.optim.Adam,
                     loss_func=criterion, metrics=accuracy_thresh)
my_learner.fit_one_cycle(1, 5e-02)
